In [1]:
import pandas as pd

file_path = r"C:\Users\charl\OneDrive\Documents\GitHub\5-CSRTT\data_performance\2025_12_12__17_50_08_659.csv"

# Skip the first 9 rows (metadata header), allow variable number of fields, handle empty cells
df = pd.read_csv(
    file_path,
    skiprows=9,
    engine='python',
    on_bad_lines='warn',
    keep_default_na=False,
    na_values=[''],
    header=0
)

# Rename columns
column_names = ["Num_line", "S", "MS", "Cat", "Num_cat", "State", "Display", "null"]
df.columns = column_names[:len(df.columns)]

df

C:\Users\charl\AppData\Local\Temp\ipykernel_4048\3597505872.py:6: ParserWarning: Skipping line 14: Expected 8 fields in line 14, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_4048\3597505872.py:6: ParserWarning: Skipping line 15: Expected 8 fields in line 15, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_4048\3597505872.py:6: ParserWarning: Skipping line 16: Expected 8 fields in line 16, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_4048\3597505872.py:6: ParserWarning: Skipping line 24: Expected 8 fields in line 24, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_4048\3597505872.py:6: ParserWarning: Skipping line 26: Expected 8 fields in line 26, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_4048\3597505872.py:6: ParserWarning: Skipping line 31: Expected 8 fields in line 31, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_4048\3597505872.py:6

,Num_line,S,MS,Cat,Num_cat,State,Display,null
0,1,0,0,Entry,0.0,Ready,NaN,NaN
1,2,0,5,Exit,0.0,Ready,Time,NaN
2,3,0,5,Entry,1.0,MagEntry,NaN,NaN
3,7,3,575,Input,1.0,On1A1,NaN,NaN
4,8,3,575,Exit,1.0,MagEntry,1,NaN
...,...,...,...,...,...,...,...,...
5001,7452,2398,760,Exit,1.0,MagEntry,1,NaN
5002,7453,2398,760,Entry,2.0,ITI2sec,NaN,NaN
5003,7454,2399,425,Input,33.0,Off1A1,NaN,NaN
5004,7455,2400,0,Exit,2.0,ITI2sec,Time,NaN


In [2]:
# Identify trials: each trial starts with "MagEntry" where Cat is "Entry" and ends with the next "MagEntry" with Cat "Entry"
# Find all indices where "MagEntry" appears in State column AND "Entry" in Cat column
magentry_indices = df[(df['State'] == 'MagEntry') & (df['Cat'] == 'Entry')].index.tolist()

# Create a list to store trial information
trials = []

# For each pair of consecutive MagEntry Entry events
for i in range(len(magentry_indices) - 1):
    trial_start_idx = magentry_indices[i]
    trial_end_idx = magentry_indices[i + 1]
    
    # Extract the trial data (from start to end, inclusive of end)
    trial_data = df.loc[trial_start_idx:trial_end_idx].copy()
    
    # Get all states/categories in this trial
    trial_states = trial_data['State'].tolist()
    
    # Store trial information using actual dataframe indices
    trials.append({
        'trial_number': i + 1,
        'start_line': trial_start_idx,
        'end_line': trial_end_idx,
        'start_idx': trial_start_idx,
        'end_idx': trial_end_idx,
        'states': trial_states,
        'num_events': len(trial_data)
    })

# Display trials summary
for trial in trials:
    print(f"Trial {trial['trial_number']}: Rows {trial['start_line']} to {trial['end_line']} ({trial['num_events']} events)")
    print(f"  States: {' -> '.join(trial['states'])}")
    print()

print(f"Total trials identified: {len(trials)}")

Trial 1: Rows 2 to 18 (17 events)
  States: MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Off1A1 -> ITI2sec -> RandomHole -> RandomHole -> hole1 -> hole1 -> Holdhole1 -> Holdhole1 -> Omission -> Omission -> TimeOut -> TimeOut -> MagEntry

Trial 2: Rows 18 to 44 (27 events)
  States: MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Off1A1 -> On1A1 -> ITI2sec -> RandomHole -> RandomHole -> hole3 -> Off1A1 -> hole3 -> Holdhole3 -> Holdhole3 -> Omission -> Omission -> TimeOut -> On1A4 -> Off1A4 -> On1A1 -> Off1A1 -> On1A1 -> Off1A1 -> On1A1 -> Off1A1 -> TimeOut -> MagEntry

Trial 3: Rows 44 to 68 (25 events)
  States: MagEntry -> On1A3 -> On1A4 -> Off1A3 -> Off1A4 -> On1A1 -> MagEntry -> ITI2sec -> Off1A1 -> On1A1 -> Off1A1 -> On1A1 -> ITI2sec -> RandomHole -> RandomHole -> hole4 -> Off1A1 -> hole4 -> Holdhole4 -> On1A2 -> On1A3 -> Holdhole4 -> CorrectResp -> CorrectResp -> MagEntry

Trial 4: Rows 68 to 96 (29 events)
  States: MagEntry -> Off1A2 -> Off1A3 -> On1A1 -> MagEntry -> ITI2sec -> Off1A

In [1]:
# Load df_659_12_12.csv and create detailed trials summary with timestamps
import pandas as pd
import json

file_path = r"C:\Users\charl\OneDrive\Documents\GitHub\5-CSRTT\df_659_12_12.csv"

# Load the data - NO skiprows since first row is header only
df = pd.read_csv(
    file_path,
    engine='python',
    on_bad_lines='warn',
    keep_default_na=False,
    na_values=[''],
    header=0
)

# Rename columns
column_names = ["Num_line", "S", "MS", "Cat", "Num_cat", "State", "Display", "null"]
df.columns = column_names[:len(df.columns)]

# Display the dataframe with actual row numbers to verify
print("Data loaded. Showing first 22 rows:")
for idx in range(min(22, len(df))):
    actual_row = idx + 2  # Convert pandas index to actual row number (row 1 = header)
    print(f"Row {actual_row}: State={df.loc[idx, 'State']}, Cat={df.loc[idx, 'Cat']}")
print(f"\nDataframe has {len(df)} rows of data\n")

# Identify trials: MagEntry with Entry
magentry_indices = df[(df['State'] == 'MagEntry') & (df['Cat'] == 'Entry')].index.tolist()
print(f"Found MagEntry+Entry at pandas indices: {magentry_indices}")
print(f"Found MagEntry+Entry at actual CSV rows: {[idx + 2 for idx in magentry_indices]}\n")

# Create detailed trials list with state information and timestamps
detailed_trials = []

for i in range(len(magentry_indices) - 1):
    trial_start_idx = magentry_indices[i]
    trial_end_idx = magentry_indices[i + 1]
    
    # Convert to actual row numbers (add 2: 1 for header row + 1 for 1-based indexing)
    actual_start_row = trial_start_idx + 2
    actual_end_row = trial_end_idx + 2
    
    # Extract trial data
    trial_data = df.loc[trial_start_idx:trial_end_idx].copy()
    
    # Create list of states with timestamps
    states_with_time = []
    for idx, row in trial_data.iterrows():
        actual_row_num = idx + 2
        states_with_time.append({
            'actual_row': actual_row_num,
            'state': row['State'],
            'cat': row['Cat'],
            'seconds': row['S'],
            'milliseconds': row['MS'],
            'display': row['Display']
        })
    
    # Store trial information with ACTUAL row numbers
    detailed_trials.append({
        'trial_number': i + 1,
        'start_row': actual_start_row,
        'end_row': actual_end_row,
        'num_events': len(trial_data),
        'states_sequence': ' -> '.join([s['state'] for s in states_with_time]),
        'states_detailed': json.dumps(states_with_time)
    })

# Create DataFrame and save to CSV
trials_df = pd.DataFrame(detailed_trials)

output_path = r"C:\Users\charl\OneDrive\Documents\GitHub\5-CSRTT\trials_summary.csv"
trials_df.to_csv(output_path, index=False)

print(f"Detailed trials summary saved to: {output_path}")
print(f"Total trials identified: {len(detailed_trials)}\n")

# Display all trials with ACTUAL row numbers
for trial in detailed_trials:
    print(f"Trial {trial['trial_number']}: Rows {trial['start_row']} to {trial['end_row']} ({trial['num_events']} events)")
    print(f"  States: {trial['states_sequence']}")
    print()

trials_df

Data loaded. Showing first 22 rows:
Row 2: State=Ready, Cat=Entry
Row 3: State=Ready, Cat=Exit
Row 4: State=MagEntry, Cat=Entry
Row 5: State=On1A1, Cat=Input
Row 6: State=MagEntry, Cat=Exit
Row 7: State=ITI2sec, Cat=Entry
Row 8: State=Off1A1, Cat=Input
Row 9: State=ITI2sec, Cat=Exit
Row 10: State=RandomHole, Cat=Entry
Row 11: State=RandomHole, Cat=Exit
Row 12: State=hole1, Cat=Entry
Row 13: State=hole1, Cat=Exit
Row 14: State=Holdhole1, Cat=Entry
Row 15: State=Holdhole1, Cat=Exit
Row 16: State=Omission, Cat=Entry
Row 17: State=Omission, Cat=Exit
Row 18: State=TimeOut, Cat=Entry
Row 19: State=TimeOut, Cat=Exit
Row 20: State=MagEntry, Cat=Entry
Row 21: State=On1A1, Cat=Input
Row 22: State=MagEntry, Cat=Exit
Row 23: State=ITI2sec, Cat=Entry

Dataframe has 5006 rows of data

Found MagEntry+Entry at pandas indices: [2, 18, 44, 68, 96, 122, 136, 152, 174, 192, 218, 232, 250, 286, 301, 319, 340, 358, 384, 397, 420, 434, 452, 468, 484, 499, 513, 527, 541, 562, 586, 605, 627, 651, 667, 686, 705

,trial_number,start_row,end_row,num_events,states_sequence,states_detailed
0,1,4,20,17,MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Of...,"[{""actual_row"": 4, ""state"": ""MagEntry"", ""cat"":..."
1,2,20,46,27,MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Of...,"[{""actual_row"": 20, ""state"": ""MagEntry"", ""cat""..."
2,3,46,70,25,MagEntry -> On1A3 -> On1A4 -> Off1A3 -> Off1A4...,"[{""actual_row"": 46, ""state"": ""MagEntry"", ""cat""..."
3,4,70,98,29,MagEntry -> Off1A2 -> Off1A3 -> On1A1 -> MagEn...,"[{""actual_row"": 70, ""state"": ""MagEntry"", ""cat""..."
4,5,98,124,27,MagEntry -> On1A4 -> Off1A4 -> On1A4 -> Off1A4...,"[{""actual_row"": 98, ""state"": ""MagEntry"", ""cat""..."
...,...,...,...,...,...,...
287,288,4921,4934,14,MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Of...,"[{""actual_row"": 4921, ""state"": ""MagEntry"", ""ca..."
288,289,4934,4955,22,MagEntry -> On1A4 -> Off1A3 -> On1A3 -> Off1A4...,"[{""actual_row"": 4934, ""state"": ""MagEntry"", ""ca..."
289,290,4955,4970,16,MagEntry -> Off1A3 -> Off1A4 -> On1A1 -> MagEn...,"[{""actual_row"": 4955, ""state"": ""MagEntry"", ""ca..."
290,291,4970,4984,15,MagEntry -> Off1A3 -> On1A1 -> MagEntry -> ITI...,"[{""actual_row"": 4970, ""state"": ""MagEntry"", ""ca..."
